In [ ]:
import pandas as pd
import numpy as np
import os
import re
import nltk

from glob import glob
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /home/maria/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/maria/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
data_path = '../Data'
twitter_data_path = glob(os.path.join(data_path, 'twitter_data_22_to_29_03_2023.csv'))[0]

In [ ]:
twitter_data_path

'../Data/twitter_data_22_to_29_03_2023.csv'

In [ ]:
# Load the Twitter data
data = pd.read_csv(twitter_data_path)
data.drop('Unnamed: 0', axis=1, inplace=True)
data.drop_duplicates(inplace=True)

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (0,5,8,13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 130694 entries, 0 to 130693
Data columns (total 21 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   public_metrics          130693 non-null  object 
 1   entities                130171 non-null  object 
 2   edit_history_tweet_ids  130693 non-null  object 
 3   referenced_tweets       108749 non-null  object 
 4   conversation_id         130693 non-null  object 
 5   text                    130693 non-null  object 
 6   author_id               130692 non-null  float64
 7   possibly_sensitive      130692 non-null  object 
 8   lang                    130692 non-null  object 
 9   created_at              130691 non-null  object 
 10  edit_controls           130691 non-null  object 
 11  reply_settings          130691 non-null  object 
 12  id                      130692 non-null  object 
 13  author                  130691 non-null  object 
 14  __twarc             

In [ ]:
data.head(3)

,public_metrics,entities,edit_history_tweet_ids,referenced_tweets,conversation_id,text,author_id,possibly_sensitive,lang,created_at,...,reply_settings,id,author,__twarc,context_annotations,in_reply_to_user_id,in_reply_to_user,attachments,geo,withheld
0,"{'retweet_count': 14, 'reply_count': 0, 'like_...","{'mentions': [{'start': 3, 'end': 17, 'usernam...",['1640956939310886913'],"[{'type': 'retweeted', 'id': '1640812385937117...",1640956939310886912,"RT @VahagnAvedian: With the pace the ""eco frie...",1.091542e+18,False,en,2023-03-29 05:59:56+00:00,...,everyone,1640956939310886912,"{'id': '1091541632212586497', 'name': 'Armine'...",{'url': 'https://api.twitter.com/2/tweets/sear...,NaN,NaN,NaN,NaN,NaN,NaN
1,"{'retweet_count': 0, 'reply_count': 1, 'like_c...","{'mentions': [{'start': 0, 'end': 16, 'usernam...",['1640956934642896896'],"[{'type': 'replied_to', 'id': '164095109079042...",1640814843073646592,@peacelover2017i @BabakTaghvaee1 Iran Russia a...,1.023852e+18,False,en,2023-03-29 05:59:55+00:00,...,everyone,1640956934642896896,"{'id': '1023851806592364544', 'name': 'ghazi',...",{'url': 'https://api.twitter.com/2/tweets/sear...,"[{'domain': {'id': '47', 'name': 'Brand', 'des...",1.520049e+18,"{'id': '1520048674684178432', 'name': 'peace l...",NaN,NaN,NaN
2,"{'retweet_count': 14, 'reply_count': 0, 'like_...","{'mentions': [{'start': 3, 'end': 19, 'usernam...",['1640956928988938240'],"[{'type': 'retweeted', 'id': '1640629789034786...",1640956928988938240,RT @REZAphotography: Profiting from the war: K...,4.858658e+07,False,en,2023-03-29 05:59:54+00:00,...,everyone,1640956928988938240,"{'id': '48586580', 'name': 'Denis Richtmann ℹ ...",{'url': 'https://api.twitter.com/2/tweets/sear...,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
data['text'][33]

'RT @CaliberEnglish: 🇦🇿🇮🇱 Today, we are going to witness a historic moment: an embassy of #Azerbaijan will open in #Israel #CaliberAz https:…'

# Preprocessing

In [ ]:
df = data.copy()

In [ ]:
# Remove URLs, user mentions, non-alphanumeric characters and hashtags from the tweets
df['text'] = df['text'].apply(lambda x: re.sub(r'http\S+', '', str(x))) # remove URLs
df['text'] = df['text'].apply(lambda x: re.sub(r'@\S+', '', str(x))) # remove user mentions
df['text'] = df['text'].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', str(x))) # remove non-alphanumeric characters
df['text'] = df['text'].apply(lambda x: re.sub(r'#\S+', '', str(x))) # remove hashtags

In [ ]:
df['text'][33]

'RT   Today we are going to witness a historic moment an embassy of Azerbaijan will open in Israel CaliberAz '

In [ ]:
# Remove punctuation and convert text to lowercase
df['text'] = df['text'].apply(lambda x: re.sub('[^\w\s]', '', x))
df['text'] = df['text'].apply(lambda x: x.lower())

In [ ]:
df['text'][33]

'rt   today we are going to witness a historic moment an embassy of azerbaijan will open in israel caliberaz '

The code below removes stop words from the text. Stop words are commonly used words in a language (such as "a", "an", "the", "is", "of", etc.) that are generally considered to be meaningless and can be safely excluded from the text analysis process. The NLTK library provides a set of predefined stop words for English language, which can be accessed using stopwords.words('english').

In [ ]:
# Remove stop words
stop_words = set(stopwords.words('english'))
df['text'] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

In [ ]:
df['text'][33]

'rt today going witness historic moment embassy azerbaijan open israel caliberaz'

In [ ]:
# Remove any remaining white space
df['text'] = df['text'].apply(lambda x: x.strip())

In [ ]:
df['text'][33]

'rt today going witness historic moment embassy azerbaijan open israel caliberaz'

In [ ]:
# Tokenize the text
df.insert(loc=6, column='text_tokens', value=df['text'].apply(lambda x: word_tokenize(x)))

In [ ]:
df['text'][33], df['text_tokens'][33]

('rt today going witness historic moment embassy azerbaijan open israel caliberaz',
 ['rt',
  'today',
  'going',
  'witness',
  'historic',
  'moment',
  'embassy',
  'azerbaijan',
  'open',
  'israel',
  'caliberaz'])

In [ ]:
df.head(3)

,public_metrics,entities,edit_history_tweet_ids,referenced_tweets,conversation_id,text,text_tokens,author_id,possibly_sensitive,lang,...,reply_settings,id,author,__twarc,context_annotations,in_reply_to_user_id,in_reply_to_user,attachments,geo,withheld
0,"{'retweet_count': 14, 'reply_count': 0, 'like_...","{'mentions': [{'start': 3, 'end': 17, 'usernam...",['1640956939310886913'],"[{'type': 'retweeted', 'id': '1640812385937117...",1640956939310886912,rt pace eco friendly regime baku building airp...,"[rt, pace, eco, friendly, regime, baku, buildi...",1.091542e+18,False,en,...,everyone,1640956939310886912,"{'id': '1091541632212586497', 'name': 'Armine'...",{'url': 'https://api.twitter.com/2/tweets/sear...,NaN,NaN,NaN,NaN,NaN,NaN
1,"{'retweet_count': 0, 'reply_count': 1, 'like_c...","{'mentions': [{'start': 0, 'end': 16, 'usernam...",['1640956934642896896'],"[{'type': 'replied_to', 'id': '164095109079042...",1640814843073646592,iran russia sent weapons armenia 56 inch chast...,"[iran, russia, sent, weapons, armenia, 56, inc...",1.023852e+18,False,en,...,everyone,1640956934642896896,"{'id': '1023851806592364544', 'name': 'ghazi',...",{'url': 'https://api.twitter.com/2/tweets/sear...,"[{'domain': {'id': '47', 'name': 'Brand', 'des...",1.520049e+18,"{'id': '1520048674684178432', 'name': 'peace l...",NaN,NaN,NaN
2,"{'retweet_count': 14, 'reply_count': 0, 'like_...","{'mentions': [{'start': 3, 'end': 19, 'usernam...",['1640956928988938240'],"[{'type': 'retweeted', 'id': '1640629789034786...",1640956928988938240,rt profiting war kremlin bypasses eu us embarg...,"[rt, profiting, war, kremlin, bypasses, eu, us...",4.858658e+07,False,en,...,everyone,1640956928988938240,"{'id': '48586580', 'name': 'Denis Richtmann ℹ ...",{'url': 'https://api.twitter.com/2/tweets/sear...,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Save the preprocessed data to a new file
output_filename = 'preprocessed_twitter_data'
df.to_csv(os.path.join(data_path, f'{output_filename}.csv'), index=False)

### Test

In [ ]:
twitter_data_path_processed = glob(os.path.join(data_path, 'preprocessed_twitter_data.csv'))[0]

In [ ]:
twitter_data_path_processed

'../Data/preprocessed_twitter_data.csv'

In [ ]:
# Load the Twitter data
data_processed = pd.read_csv(twitter_data_path_processed)

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (4,8,13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
data_processed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130694 entries, 0 to 130693
Data columns (total 22 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   public_metrics          130693 non-null  object 
 1   entities                130171 non-null  object 
 2   edit_history_tweet_ids  130693 non-null  object 
 3   referenced_tweets       108749 non-null  object 
 4   conversation_id         130693 non-null  object 
 5   text                    129554 non-null  object 
 6   text_tokens             130694 non-null  object 
 7   author_id               130692 non-null  float64
 8   possibly_sensitive      130692 non-null  object 
 9   lang                    130692 non-null  object 
 10  created_at              130691 non-null  object 
 11  edit_controls           130691 non-null  object 
 12  reply_settings          130691 non-null  object 
 13  id                      130692 non-null  object 
 14  author              

In [ ]:
data_processed.head(3)

,public_metrics,entities,edit_history_tweet_ids,referenced_tweets,conversation_id,text,text_tokens,author_id,possibly_sensitive,lang,...,reply_settings,id,author,__twarc,context_annotations,in_reply_to_user_id,in_reply_to_user,attachments,geo,withheld
0,"{'retweet_count': 14, 'reply_count': 0, 'like_...","{'mentions': [{'start': 3, 'end': 17, 'usernam...",['1640956939310886913'],"[{'type': 'retweeted', 'id': '1640812385937117...",1640956939310886912,rt pace eco friendly regime baku building airp...,"['rt', 'pace', 'eco', 'friendly', 'regime', 'b...",1.091542e+18,False,en,...,everyone,1640956939310886912,"{'id': '1091541632212586497', 'name': 'Armine'...",{'url': 'https://api.twitter.com/2/tweets/sear...,NaN,NaN,NaN,NaN,NaN,NaN
1,"{'retweet_count': 0, 'reply_count': 1, 'like_c...","{'mentions': [{'start': 0, 'end': 16, 'usernam...",['1640956934642896896'],"[{'type': 'replied_to', 'id': '164095109079042...",1640814843073646592,iran russia sent weapons armenia 56 inch chast...,"['iran', 'russia', 'sent', 'weapons', 'armenia...",1.023852e+18,False,en,...,everyone,1640956934642896896,"{'id': '1023851806592364544', 'name': 'ghazi',...",{'url': 'https://api.twitter.com/2/tweets/sear...,"[{'domain': {'id': '47', 'name': 'Brand', 'des...",1.520049e+18,"{'id': '1520048674684178432', 'name': 'peace l...",NaN,NaN,NaN
2,"{'retweet_count': 14, 'reply_count': 0, 'like_...","{'mentions': [{'start': 3, 'end': 19, 'usernam...",['1640956928988938240'],"[{'type': 'retweeted', 'id': '1640629789034786...",1640956928988938240,rt profiting war kremlin bypasses eu us embarg...,"['rt', 'profiting', 'war', 'kremlin', 'bypasse...",4.858658e+07,False,en,...,everyone,1640956928988938240,"{'id': '48586580', 'name': 'Denis Richtmann ℹ ...",{'url': 'https://api.twitter.com/2/tweets/sear...,NaN,NaN,NaN,NaN,NaN,NaN
